
#            TP Identification de paramètres d'un robot RRR par techniques des moindres carrées #



Soit le robot RRR correspondant à la figure suivante:

![alternative text](robot-rrr3-sol.png)

- La configuration du robot est défini par le vecteur $q = (q_1, q_2, q_3)^t$.

- La situation de l'outil est défini par le vecteur $X = (x, y, z)^t$ .

## Objectif
L'objectif est d'identifier certains paramètres géométriques du robot plan RRR.

A l'aide d'une caméra, on a mesuré le vecteur $X$ pour un ensemble de configurations $q$. On considère que toutes les mesures sont exactes (pas d'erreurs liées au système de mesure).

Les résultats se trouvent dans les fichier *mesuresX.dat*. Chaque ligne comporte une mesure avec la convention suivante pour les 6 valeurs:

$q_1$, $q_2$, $q_3$, x, y, z sur chaque ligne (format %12.6f pour chaque valeur). Les longueurs sont exprimés en mètre.

- A: Dans un premier temps on veut identifier les longueurs des liaisons $a$, $b$, $c$ du robot RRR en considérant que les valeurs des angles $q_i$ sont exactes.

Fichier de mesure: *mesures1.dat*


## Travail

Modéliser le problème comme un problème d'optimisation

Implémenter votre problème et le résoudre 


In [2]:
#################################################   
# Import 
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
import scipy
import time
from random import *
from numpy.linalg import inv,cond
from scipy.optimize import root
from scipy.optimize import least_squares

In [3]:
#################################################   
# Calcul du MGD du robot RRR
# INPUT:  q = vecteur de configuration (deg, deg, deg)
# OUTPUT: Xc = vecteur de situation = position3D = (x,y,z) 
def mgd(qdeg,a,b,c):
    qrad=np.deg2rad(qdeg)
    # introduction d'un eerreur su run qi
    # erreur= 5 (en degré)
    # erg = np.deg2rad(erreur)
    # qrad[i] = qi + erg
    c1= np.cos(qrad[0])
    s1=np.sin(qrad[0])
    c23= np.cos(qrad[2]+qrad[1])
    s23= np.sin(qrad[2]+qrad[1])
    c2=np.cos(qrad[1])
    s2=np.sin(qrad[1])
    x= a*c1*c2 + b*c1*c23 
    y= a*s1*c2 + b*s1*c23 
    z= a*s2 + b*s23 + c 
    Xd=[x,y,z]
    return Xd

#

In [4]:
# Lecture mesures de fichier

with open("mesures1.dat", "r") as f:
    line= list()

    donnees= list()
    xdata= list()
    qdata= list()
    q1data= list()
    q2data= list()
    q3data= list()
    for line in f:
        if "#" in line:
            # on saute la ligne
            continue
        data = line.split()
        donnees.append((float(data[0]), float(data[1]),float(data[2]), float(data[3]), float(data[4]),float(data[5])))
        xdata.append([float(data[3]), float(data[4]),float(data[5])])
        qdata.append([float(data[0]), float(data[1]),float(data[2])])
        q1data.append(float(data[0]))
        q2data.append(float(data[1]))
        q3data.append(float(data[2]))
#donnees
#xdata


In [5]:
import numpy as np

# Fonction pour calculer les résidus
def residuals(params, *args):
    a, b, c = params
    xdata, qdata = args
    residuals = []
    for i in range(len(xdata)):
        X_predicted = np.array(mgd(qdata[i], a, b, c))  # Convertir en tableau NumPy
        X_actual = np.array(xdata[i])  # Convertir en tableau NumPy
        residuals.extend(X_predicted - X_actual)
    return residuals

# Construction de la matrice A
def build_A(qdata):
    A = []
    for q in qdata:
        A_i = [mgd(q, 1.0, 0.0, 0.0), mgd(q, 0.0, 1.0, 0.0), mgd(q, 0.0, 0.0, 1.0)]
        A.extend(A_i)
    return np.array(A)

# Initialisation des paramètres a, b, et c
initial_guess = np.array([1.0, 1.0, 1.0])

# Construction de la matrice A
A = build_A(qdata)

# Construction du vecteur b
b = np.array([item for sublist in xdata for item in sublist])

# Calcul des paramètres optimaux à l'aide de la formule des moindres carrés
params_optimal = np.linalg.inv(np.transpose(A).dot(A)).dot(np.transpose(A)).dot(b)

# Les valeurs optimales de a, b, et c sont dans params_optimal
a_optimal, b_optimal, c_optimal = params_optimal

# Affichage des résultats
print("Valeurs optimales de a, b et c :")
print("a_optimal =", a_optimal)
print("b_optimal =", b_optimal)
print("c_optimal =", c_optimal)


Valeurs optimales de a, b et c :
a_optimal = 2.1756272001307
b_optimal = 2.145270178607535
c_optimal = 1.350996400334573


### Test du MGD après identification

Vérifier que votre identification des longueurs $a$, $b$ et $c$ est correcte.

In [6]:
# Utiliser les valeurs optimales de a, b et c identifiées précédemment
a_optimal, b_optimal, c_optimal = result.x

# Générer des configurations q pour la simulation (par exemple, aléatoirement)
num_samples = len(xdata)  # Utiliser le nombre de mesures réelles comme nombre d'échantillons

# Simuler les mesures avec les paramètres identifiés
x_simulated = [mgd(q, a_optimal, b_optimal, c_optimal) for q in qdata[:num_samples]]

# Comparaison des mesures simulées et du sous-ensemble des mesures réelles
x_reel = xdata[:num_samples]  # Utiliser un sous-ensemble de mesures réelles

# Calcul de l'erreur RMSE
rmse = np.sqrt(np.mean((np.array(x_simulated) - np.array(x_reel))**2))

# Visualisation (facultatif)
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*zip(*x_simulated), label='Mesures simulées', c='r', marker='o')
ax.scatter(*zip(*x_reel), label='Mesures réelles', c='b', marker='^')
plt.legend()
plt.title(f'RMSE = {rmse:.4f}')
plt.show()

# Affichage du RMSE
print(f"RMSE entre les mesures simulées et réelles : {rmse:.4f}")


NameError: name 'result' is not defined

## Utilisation de scipy optimize

Retrouver vos résultats en utilisant directement les fonctions de scipy optimize

In [7]:
from scipy.optimize import least_squares

# Fonction objectif à minimiser
def objective(params, *args):
    a, b, c = params
    xdata, qdata = args
    error = []
    for i in range(len(xdata)):
        X_predicted = np.array(mgd(qdata[i], a, b, c))  # Convertir en tableau NumPy
        X_actual = np.array(xdata[i])  # Convertir en tableau NumPy
        error.extend(X_predicted - X_actual)
    return error


# Initialisation des paramètres a, b, et c
initial_guess = [1.0, 1.0, 1.0]

# Appel de la fonction least_squares pour résoudre le problème d'optimisation
result = least_squares(objective, initial_guess, args=(xdata, qdata))

# Les valeurs optimales de a, b, et c sont dans result.x
a_optimal, b_optimal, c_optimal = result.x

# Affichage des résultats
print("Valeurs optimales de a, b et c :")
print("a_optimal =", a_optimal)
print("b_optimal =", b_optimal)
print("c_optimal =", c_optimal)




Valeurs optimales de a, b et c :
a_optimal = 3.9970453232848104
b_optimal = 2.4995117574435537
c_optimal = 3.2966481231049403
